### Nombre: Elena Jaramillo Rosado

## Segundo Clasificador TFM

Este segundo clasificador toma los datos del dataset __dataset_more_labeled.csv__ y predice a partir de las columnas 'bio' e 'interest' si un usuario especificará o no su género. 

En este caso, se seguirá un procedimiento de aprendizaje supervisado clásico como es el uso de árboles de decisión (RandomForest)

In [27]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from imblearn.over_sampling import SMOTE

In [ ]:
# Ruta al dataset
path_df = './datasets/dataset_more_labeled.csv'

df = pd.read_csv(path_df, sep = ',', header = 0)
print(df.head(10))

    age                                                bio               job  \
0  18.0                    ::anatomical_heart: :venezuela:  ['NotSpecified']   
1  25.0  i don't know what i want or what i'm looking f...  ['NotSpecified']   
2  20.0             to the insta better: :sparkling_heart:  ['NotSpecified']   
3  19.0  :libra: :woman_fairy_light_skin_tone::performi...  ['NotSpecified']   
4  20.0  if you are the one in the last photo, go ahead...  ['NotSpecified']   
5  24.0  discovering new places; chill out plans and th...  ['NotSpecified']   
6  23.0                                              :ice:  ['NotSpecified']   
7  21.0                                         best insta  ['NotSpecified']   
8  26.0  venezuela :venezuela::red_heart: :round_pushpi...  ['NotSpecified']   
9  18.0   talk to me on instagram < i put a random age ...  ['NotSpecified']   

                           school              anthem_title  \
0                             NaN                       

Para simplificar esta tarea, vamos a crear una columna nueva llamada 'gender-specified' que sólo tomará dos valores: 

- 0 si 'gender' es distinto a 'NotSpecified'
- 1 si 'gender' es igual a 'NotSpecified'

In [29]:
# Crear la columna 'gender-specified'
df['gender-specified'] = df['gender'].apply(lambda x: 1 if x == 'NotSpecified' else 0)

Empezamos a preparar los datos para usa el modelo. 

In [ ]:
# Vectorización con TF-IDF
vectorizer = TfidfVectorizer(
    max_features=500, 
    stop_words='english',  # No se eliminan stopwords por idioma mixto
    ngram_range=(1, 2)  # Usar unigramas y bigramas para contexto adicional
)

In [31]:
df['bio'] = df['bio'].fillna('')
df['interest_text'] = df['interest_text'].fillna('')

X = vectorizer.fit_transform(df['bio'] + ' ' + df['interest_text'])

In [ ]:
#Modelo predictivo
y = df['gender-specified']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Ya que aún habiendo usado el Primer Clasificador, sigue habiendo un desbalanceo entre los registros con 'gender' especificado y los que no, vamos a usar técnica de __oversampling__. En este caso, he optado por SMOTE.

In [33]:
# Usar SMOTE para balancear las clases (opcional)
smote = SMOTE(random_state=42)
X_train_sm, y_train_sm = smote.fit_resample(X_train, y_train)

In [34]:
# Usar RandomForestClassifier como modelo base
model = RandomForestClassifier(
    n_estimators=200,  # Más árboles para mayor estabilidad
    max_depth=None,    # Permitir que crezca completamente
    random_state=24
)

model.fit(X_train_sm, y_train_sm)

# Evaluar el modelo
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.62      0.46      0.53       475
           1       0.69      0.81      0.75       708

    accuracy                           0.67      1183
   macro avg       0.65      0.63      0.64      1183
weighted avg       0.66      0.67      0.66      1183

